In [ ]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt
!pip install insightface

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh -y

# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

# CheckPoint

In [ ]:
# Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/198530'
model_name = 'JuggernautXL.safetensors'

%cd $checkpoints
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Lora

In [ ]:
# Install a LoRA in permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

%cd /kaggle/working/ComfyUI/models/loras
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Launch With RemoteMoe

In [ ]:
# Starting with RemoteMoe

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python /kaggle/working/ComfyUI/main.py & ssh -R 80:127.0.0.1:8188 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Launch with Pinggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

In [ ]:
import os
import requests

# Define the URL and the destination path
url = "https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip_gan.pth?download=true"
dest_folder = "/kaggle/working/ComfyUI/custom_nodes/ComfyUI_wav2lip/Wav2Lip/checkpoints"
dest_file = os.path.join(dest_folder, "wav2lip_gan.pth")

# Create the destination folder if it doesn't exist
os.makedirs(dest_folder, exist_ok=True)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(dest_file, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded and saved to {dest_file}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


In [ ]:
!pip install py7zr


In [ ]:
import os
import shutil
import py7zr

# Paths
fix_folder = "/kaggle/working/ComfyUI/custom_nodes/ComfyUI_wav2lip/ComfyUI-VideoHelperSuite-fix"
zip_file = os.path.join(fix_folder, "ComfyUI-VideoHelperSuite.7z")
temp_extract_folder = "/kaggle/working/temp_ComfyUI_VideoHelperSuite"
existing_node_folder = "/kaggle/working/ComfyUI/custom_nodes/ComfyUI-VideoHelperSuite"

# Verify if the .7z file exists
if not os.path.isfile(zip_file):
    raise FileNotFoundError(f"The file {zip_file} does not exist.")

# Extract the .7z file to a temporary location
os.makedirs(temp_extract_folder, exist_ok=True)

with py7zr.SevenZipFile(zip_file, mode='r') as archive:
    archive.extractall(path=temp_extract_folder)

# Delete the existing ComfyUI-VideoHelperSuite node
if os.path.exists(existing_node_folder):
    shutil.rmtree(existing_node_folder)

# Move the extracted contents to replace the existing node
extracted_folder_name = os.listdir(temp_extract_folder)[0]  # Assumes a single top-level folder
extracted_folder_path = os.path.join(temp_extract_folder, extracted_folder_name)
shutil.move(extracted_folder_path, existing_node_folder)

# Clean up the temporary extraction folder
shutil.rmtree(temp_extract_folder)

print("Replacement complete. Restart ComfyUI to apply the changes.")
